In [1]:
# Initial code for processing data from CMIP6 ensemble for senior thesis project
# Author: Nathan Erickson
# Date: 9/13/2021
# Coded with Python 3.8.10

In [2]:
# Directory management

import os

#dir = '/chinook2/nathane1/Thesis/CMIP6/'
#model = 'CESM2'
#path = dir + model
#os.chdir(path)

In [3]:
# Import other important modules

import numpy as np
import pandas as pd
import xarray as xr
import cftime as cf
import warnings
import eli_processing

warnings.simplefilter("ignore","SerializationWarning:")

In [4]:
# Read in files from working directory
# Read in model data

data = xr.open_dataset('sst.mnmean.nc', decode_times = False)

# Read in land mask

land_mask = xr.open_dataset(f'/chinook2/nathane1/Thesis/CMIP6/ACCESS-CM2/sftlf_fx_ACCESS-CM2_historical_r1i1p1f1_gn.nc')

In [5]:
data

<xarray.Dataset>
Dimensions:    (lat: 89, lon: 180, time: 2012, nbnds: 2)
Coordinates:
  * lat        (lat) float32 88.0 86.0 84.0 82.0 ... -82.0 -84.0 -86.0 -88.0
  * lon        (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * time       (time) float64 1.972e+04 1.975e+04 ... 8.09e+04 8.093e+04
Dimensions without coordinates: nbnds
Data variables:
    time_bnds  (time, nbnds) float64 ...
    sst        (time, lat, lon) float32 ...
Attributes: (12/37)
    climatology:               Climatology is based on 1971-2000 SST, Xue, Y....
    description:               In situ data: ICOADS2.5 before 2007 and NCEP i...
    keywords_vocabulary:       NASA Global Change Master Directory (GCMD) Sci...
    keywords:                  Earth Science > Oceans > Ocean Temperature > S...
    instrument:                Conventional thermometers
    source_comment:            SSTs were observed by conventional thermometer...
    ...                        ...
    creator_url_original:      https://www.ncei.noaa.gov
    license:                   No constraints on data access or use
    comment:                   SSTs were observed by conventional thermometer...
    summary:                   ERSST.v5 is developed based on v4 after revisi...
    dataset_title:             NOAA Extended Reconstructed SST V5
    data_modified:             2021-09-07

In [6]:
ts = data['sst']
lon = data['lon']
lat = data['lat']

In [7]:
datetimes = pd.date_range('1854-01-01','2021-08-01',freq='MS').strftime('%m-%Y')
np.array(datetimes)

array(['01-1854', '02-1854', '03-1854', ..., '06-2021', '07-2021',
       '08-2021'], dtype=object)

In [8]:
# Define functions for calculation of ELI

# Slice data to include only between 5 S - 5 N and only equatorial Pacific

ts_pac = ts.sel(lon=slice(115,290),lat=slice(5,-5))
ts_tropics = ts.sel(lat=slice(5,-5))

In [9]:
ts_tropics

<xarray.DataArray 'sst' (time: 2012, lat: 5, lon: 180)>
[1810800 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 4.0 2.0 0.0 -2.0 -4.0
  * lon      (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 350.0 352.0 354.0 356.0 358.0
  * time     (time) float64 1.972e+04 1.975e+04 1.978e+04 ... 8.09e+04 8.093e+04
Attributes:
    long_name:     Monthly Means of Sea Surface Temperature
    units:         degC
    var_desc:      Sea Surface Temperature
    level_desc:    Surface
    statistic:     Mean
    dataset:       NOAA Extended Reconstructed SST V5
    parent_stat:   Individual Values
    actual_range:  [-1.8     42.32636]
    valid_range:   [-1.8 45. ]

In [10]:
threshold_temp = np.nanmean(ts_tropics)
#np.nanmean(ts_pac[48]['lon'].where(ts_pac[48]>threshold_temp))

In [11]:
# Binary spatial mask (1 = TS > threshold temperature (which is what???); 0 = TS < threshold temperature)

time = data['time']
monthly_averages = []
monthly_ELI = []
for t in range(0,len(ts_pac)):
    monthly_avg = np.nanmean(ts_tropics[t])
    monthly_averages.append(monthly_avg)# Find average SST of all points
    threshold_temp = monthly_avg
    ELI_points = ts_pac[t]['lon'].where(ts_pac[t]>threshold_temp)
    ELI = np.nanmean(ELI_points)
    monthly_ELI.append(ELI)

In [ ]:
monthly_ELI

In [13]:
from datetime import datetime
datetime_series = pd.date_range(start='01-1854',periods=len(monthly_ELI),freq='M')
redatetime = []
for date in datetime_series:
    redatetime.append(datetime.strftime(date,'%Y-%m'))

In [ ]:
redatetime

In [15]:
ERSST = pd.DataFrame(data=monthly_ELI, 
                     index = redatetime,
                     columns = ['ERSST_v5'])
ERSST.to_csv('ERSST_v5_data.csv')

In [16]:
eli_table = pd.read_csv('ERSST_v5_data.csv', index_col = 'Unnamed: 0')
eli_table

ERSST_v5
1854-01  158.62827
1854-02  159.30386
1854-03  154.63530
1854-04  154.60228
1854-05  160.55770
...            ...
2021-04  148.66216
2021-05  154.55865
2021-06  160.33493
2021-07  163.07762
2021-08  163.39726

[2012 rows x 1 columns]

In [34]:
len(eli_table)/12

167.66666666666666

In [63]:
 #Select only ELI rows from DJF
djf_table = eli_table.loc[[(pd.Index(eli_table.index)[time][5:] == '12') or (pd.Index(eli_table.index)[time][5:] == '01') or (pd.Index(eli_table.index)[time][5:] == '02') for time in range(len(eli_table.index))]]
#print(djf_table)
djf_listed = []
for model in djf_table.columns[range(len(djf_table.T))]:
    djf_listed.append(model)

# Take means of sequential DJF seasons
import numpy as np
mean_list = []
for model in djf_table.columns[range(len(djf_table.T))]:
    for month in range(len(djf_table)):
        mean_list.append(np.mean(djf_table[model][month:month+3]))
#print(len(mean_list))
        
# Remove final element from each model list to avoid bad averages at end of period
djf_naiveaverages = mean_list[2::3]
#print(djf_naiveaverages)
# Line below is not appropriately generalized
djf_averages = [djf_naiveaverages[x:x+int(len(mean_list)/3)] 
                for x in range(0,len(eli_table.columns),1)] 
#else:
#print(djf_averages)
if (int(len(mean_list)/3) == float(len(mean_list)/3)):
    for lonely_december in djf_averages:
        del lonely_december[len(lonely_december)-1]

# Create a new datetime-like index for a finalized dataframe with only DJF data
djf_dates_start = pd.date_range(start='1854',periods=len(djf_averages[0]),freq='Y')
djf_dates_end = pd.date_range(start='1855',periods=len(djf_averages[0]),freq='Y')
djf_datetime = []
for date in range(len(djf_averages[0])):
    djf_datetime.append(f'{str(djf_dates_start[date])[0:4]}-{str(djf_dates_end[date])[0:4]}')

In [ ]:
djf_datetime

In [67]:
djf_data = pd.DataFrame(data = djf_averages,index = ['ERSST_v5'],columns = djf_datetime)
djf_data.T

ERSST_v5
1854-1855  157.437703
1855-1856  164.050207
1856-1857  154.700113
1857-1858  164.951260
1858-1859  157.113673
...               ...
2016-2017  156.339447
2017-2018  154.990253
2018-2019  161.846820
2019-2020  158.957753
2020-2021  152.404207

[167 rows x 1 columns]

In [68]:
djf_data.T.to_csv('djf_ERSSTv5.csv')

In [ ]:
def djf_average_construct(eli_table):
    #Select only ELI rows from DJF
    djf_table = eli_table.loc[[(pd.Index(eli_table.index)[time][5:] == '12') or (pd.Index(eli_table.index)[time][5:] == '01') or (pd.Index(eli_table.index)[time][5:] == '02') for time in range(len(eli_table.index))]]
    djf_listed = []
    for model in djf_table.columns[range(len(djf_table.T))]:
        djf_listed.append(model)

    # Take means of sequential DJF seasons
    import numpy as np
    mean_list = []
    for model in djf_table.columns[range(len(djf_table.T))]:
        for month in range(len(djf_table)):
            mean_list.append(np.mean(djf_table[model][month:month+3]))

    # Remove final element from each model list to avoid bad averages at end of period
    djf_naiveaverages = mean_list[2::3]
    # Line below is not appropriately generalized
    djf_averages = [djf_naiveaverages[x:x+len(djf_naiveaverages[1])] 
                    for x in range(0,len(djf_naiveaverages),len(djf_naiveaverages[1]))] 
    print(djf_averages)
    for lonely_december in djf_averages:
            del lonely_december[len(lonely_december)-1]

    # Create a new datetime-like index for a finalized dataframe with only DJF data
    djf_dates_start = pd.date_range(start='1850',periods=len(djf_averages[0]),freq='Y')
    djf_dates_end = pd.date_range(start='1851',periods=len(djf_averages[0]),freq='Y')
    djf_datetime = []
    for date in range(len(djf_dates)):
        djf_datetime.append(f'{str(djf_dates_start[date])[0:4]}-{str(djf_dates_end[date])[0:4]}')

In [ ]:
djf_average_construct(ERSST_data)

In [ ]:
# Import modules and set up for plotting

#import matplotlib.pyplot as plt
#import cartopy as cp


#ELI_trend = np.polyfit(time.astype(float),monthly_ELI,1)
#trend = np.poly1d(ELI_trend)
#plt.figure(figsize = (16,6))
#ax = plt.axes()
#ax.plot(np.array(datetimes),monthly_ELI)
#ax.grid()
#ax.set_title(f'ERSSTv5 ELI Time Series', size = 18, loc = 'center')